In [1]:
import openai
import os
from dotenv import load_dotenv
import chromadb


In [2]:
load_dotenv()

True

In [3]:
OPEN_AI_API_KEY = os.getenv("OPENAI_API_KEY")

In [4]:
openai.api_key = OPEN_AI_API_KEY

In [5]:
CHROMA_DB_PATH = "./../data_collecting/chroma_db"

In [6]:
client = chromadb.PersistentClient(path=CHROMA_DB_PATH)
collection = client.get_or_create_collection(name="python_data")

In [7]:
print(len(collection.get()["metadatas"]))

93929


In [17]:
chat_model = "gpt-4o-mini"

In [39]:
import tiktoken
tokenizer = tiktoken.encoding_for_model("text-embedding-ada-002")

In [81]:
prompt_model = """
    ### Context:
    {context}

    ### Python version: 
    {python_version}
    
    ** Instructions **
    - If user asks you to generate code and by using context you cannot do it, then generate it on your own
    - If user doesn't ask to generate code and the context does not contain answer for query answet based on your knowledge.
    - If the user's question does not specify Python, rephrase it internally as a Python-related question before answering.
    - If there is a code in your output explain this code to the user step by step
    - Do not answer any other question than about python programming language
    - If topic is complex provide summary at the end of your answer
    - Do not make up any information
    - Provide consise and structured answer

    ### Summary of previous conversation:
    {summary}

    ### Recent conversation history:
    
    """


In [41]:
def get_openai_embedding(text):
    response = openai.embeddings.create(
        input=[text],
        model="text-embedding-ada-002"
    )
    return response.data[0].embedding

def retrieve_documents(query, python_version, top_k=7):
    query_embedding = get_openai_embedding(query)

    results = collection.query(
        query_embeddings=[query_embedding],
        n_results=top_k,
        where={"version": python_version}
    )

    return results["documents"][0] if "documents" in results and results["documents"] else []

def generate_response(query, retrieved_docs, python_version):
    context = "\n\n".join(retrieved_docs)
    
    prompt = prompt_model.format(context = context,query = query,python_version = python_version)
    client = openai.Client()
    response = client.chat.completions.create(
        model=chat_model,
        temperature=0.3,
        messages=[{"role": "system", "content": "You are python expert and you provide answer only based on given context."},
                  {"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [70]:
class SummaryBufferMemory:
    def __init__(self, max_tokens=3500, window_size=5, model="gpt4o-mini"):
        self.history = []
        self.summary = ""
        self.max_tokens = max_tokens
        self.window_size = window_size
        self.model = model

    def add_interaction(self, query, assistant_output):
        self.history.append({"user": query, "assistant": assistant_output})
        if self.get_token_count() > self.max_tokens:
            self.summarize_history()

    def get_token_count(self):
        text = " ".join([f"{h['user']} {h['assistant']}" for h in self.history])
        return len(tokenizer.encode(text))

    def summarize_history(self):
        conversation_text = "\n".join(
            f"User: {h['user']}\nAssistant: {h['assistant']}" for h in self.history[:-self.window_size]
        )

        summarization_prompt = f"""
        Your task is to summarize the following conversation between a user and an assistant.
        - Focus ONLY on key technical details (technologies, libraries, coding languages, user's project specifics).
        - Omit greetings, general questions, or small talk.
        - Limit your summary to 7-10 concise sentences.

        Conversation:
        {conversation_text}

        Summary:
        """

        response = openai.ChatCompletion.create(
            model=self.model,
            messages=[{"role": "user", "content": summarization_prompt}],
            temperature=0.2
        )

        self.summary = response.choices[0].message.content

        self.history = self.history[-self.window_size:]

    def get_prompt(self, query, context, python_version="3.10"):
        prompt = prompt_model.format(context = context,query = query,python_version = python_version, summary=self.summary)

        for msg in self.history:
            prompt += f"User: {msg['user']}\nAssistant: {msg['assistant']}\n"

        prompt += f"User: {query}\nAssistant:"

        return prompt


def generate_with_memory(query, context, memory, python_version="3.11"):
    context = "\n\n".join(context)
    client = openai.Client()
    prompt = memory.get_prompt(query, context, python_version)
    print(prompt)
    
    response = client.chat.completions.create(
        model=chat_model,
        temperature=0.3,
        messages=[
            {"role": "system", "content": "You are python expert and you provide answer only based on given context."},
            {"role": "user", "content": prompt}
        ]
    )

    assistant_response = response.choices[0].message.content
    memory.add_interaction(query, assistant_output=assistant_response)

    return assistant_response

In [43]:
retrieved_docs = retrieve_documents("How can i open file?", "3.10", 8)
answer = generate_response("How can i open file?", retrieved_docs, "3.10")

KeyError: 'summary'

In [19]:
answer

'To open a file in Python, you can use the built-in `open()` function. Here’s a simple example of how to use it:\n\n```python\nfile = open(\'example.txt\', \'r\')\n```\n\n### Explanation:\n1. **`open()` Function**: This function is used to open a file. It takes two main arguments:\n   - The first argument is the name of the file you want to open (in this case, `\'example.txt\'`).\n   - The second argument is the mode in which you want to open the file. In this example, `\'r\'` stands for "read" mode, which means you want to read the contents of the file.\n\n2. **File Object**: The `open()` function returns a file object, which you can use to read from or write to the file.\n\n### Summary:\nTo open a file in Python, use the `open()` function with the appropriate file name and mode. In this example, we opened a file named `\'example.txt\'` in read mode.'

In [95]:
memory = SummaryBufferMemory(max_tokens=2000, window_size=5, model=chat_model)


In [96]:
query = "How to implement decorators in python?"
retrieved_docs = retrieve_documents(query, "3.10", top_k=7)

answer = generate_with_memory(query, retrieved_docs, memory, "3.10")



    ### Context:
    ## Functions and decorators Â¶


## Patch Decorators Â¶


## Nesting Patch Decorators Â¶



If you want several patches in place for multiple test methods the obvious way
is to apply the patch decorators to every method. This can feel like unnecessary
repetition. For Python 2.6 or more recent you can use patch() (in all its
various forms) as a class decorator. This applies the patches to all test
methods on the class. A test method is identified by methods whose names start
with test :
`patch()`
`patch()`
`test`
`test`
>>> @patch ( 'mymodule.SomeClass' ) ... class MyTest ( unittest . TestCase ): ... ... def test_one ( self , MockSomeClass ): ... self . assertIs ( mymodule . SomeClass , MockSomeClass ) ... ... def test_two ( self , MockSomeClass ): ... self . assertIs ( mymodule . SomeClass , MockSomeClass ) ... ... def not_a_test ( self ): ... return 'something' ... >>> MyTest ( 'test_one' ) . test_one () >>> MyTest ( 'test_two' ) . test_two () >>> MyTest ( 'test_

In [97]:
answer

'To implement decorators in Python, you typically define a function that takes another function as an argument and returns a new function that usually extends or modifies the behavior of the original function. Here’s a step-by-step explanation:\n\n1. **Define the Decorator Function**: Create a function that takes a function as an argument.\n2. **Define an Inner Function**: Inside the decorator, define a nested function that will wrap the original function.\n3. **Call the Original Function**: Inside the inner function, you can call the original function and add any additional behavior before or after this call.\n4. **Return the Inner Function**: The outer function should return the inner function.\n\nHere’s a simple example:\n\n```python\ndef my_decorator(func):\n    def wrapper():\n        print("Something is happening before the function is called.")\n        func()\n        print("Something is happening after the function is called.")\n    return wrapper\n\n@my_decorator\ndef say_hel

In [99]:
query = "What is wrong with you?"
retrieved_docs = retrieve_documents(query, "3.10", top_k=7)

answer = generate_with_memory(query, retrieved_docs, memory, "3.10")


    ### Context:
    ## ANY Â¶


## help Â¶


`



()



"`



## Babyl Â¶


## MH Â¶


    ### Python version: 
    3.10
    
    ** Instructions **
    - If user asks you to generate code and by using context you cannot do it, then generate it on your own
    - If user doesn't ask to generate code and the context does not contain answer for query answet based on your knowledge.
    - If the user's question does not specify Python, rephrase it internally as a Python-related question before answering.
    - If there is a code in your output explain this code to the user step by step
    - Do not answer any other question than about python programming language
    - If topic is complex provide summary at the end of your answer
    - Do not make up any information
    - Provide consise and structured answer

    ### Summary of previous conversation:
    

    ### Recent conversation history:
    
    User: How to implement decorators in python?
Assistant: To implement decorators in Pyth

In [100]:
answer

"I'm here to assist you with Python programming questions. If you have a specific query or need help with Python code, please let me know!"